In [35]:
import os

In [36]:
dataset = "RoboFlow-2_relabel"
run_dir = "yolo11n-RoboFlow-2-no-medical"
train_dir = "predict"

# get the current working directory
script_dir = os.getcwd()
# get the parent directory
src_dir = os.path.dirname(script_dir)

# get the dataset directory
prj_dir = os.path.dirname(src_dir)
datasets_dir = os.path.join(prj_dir, "datasets", "extracted")
data_dir = os.path.join(datasets_dir, dataset)

# get the results directory
detect_dir = os.path.join(src_dir, "training", "runs", "detect")
results_dir = os.path.join(detect_dir, run_dir, train_dir)

if not os.path.exists(results_dir):
    print(f"Results directory {results_dir} does not exist.")

if not os.path.exists(data_dir):
    print(f"Data directory {data_dir} does not exist.")

In [37]:
# sorted list of dataset test labels files
data_labels_dir = os.path.join(data_dir, "test", "labels")
# txt files in listdir
data_labels_files = sorted([f for f in os.listdir(data_labels_dir) if f.endswith('.txt')])

# sorted list of results labels files
results_labels_dir = os.path.join(results_dir, "labels")
# txt files in listdir
results_labels_files = sorted([f for f in os.listdir(results_labels_dir) if f.endswith('.txt')])

In [38]:
from sklearn.cluster import DBSCAN
import numpy as np

def find_clusters(labels_files, labels_dir):
    clusters = []
    for file in labels_files:
        file_path = os.path.join(labels_dir, file)
        with open(file_path, 'r') as f:
            lines = f.readlines()
        bboxes = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            bboxes.append((class_id, x_center, y_center, width, height))
        if not bboxes:
            continue
        bboxes = np.array(bboxes)
        for class_id in np.unique(bboxes[:, 0]):
            class_bboxes = bboxes[bboxes[:, 0] == class_id]
            if len(class_bboxes) < 2:
                continue
            centers = class_bboxes[:, 1:3]
            widths = class_bboxes[:, 3]
            heights = class_bboxes[:, 4]
            distance_matrix = np.zeros((len(class_bboxes), len(class_bboxes)))
            for i in range(len(class_bboxes)):
                for j in range(len(class_bboxes)):
                    if i == j:
                        continue
                    distance_horizontal = abs(centers[i][0] - centers[j][0])
                    distance_vertical = abs(centers[i][1] - centers[j][1])
                    distance_matrix[i][j] = max(
                        0,
                        distance_horizontal - widths[i] / 2 - widths[j] / 2,
                        distance_vertical - heights[i] / 2 - heights[j] / 2
                    )
            db = DBSCAN(eps=0.7, min_samples=2, metric='precomputed')
            labels = db.fit_predict(distance_matrix)
            unique_labels = set(labels)
            for label in unique_labels:
                if label == -1:
                    continue
                cluster_indices = np.where(labels == label)[0]
                cluster_bboxes = class_bboxes[cluster_indices]
                clusters.append(cluster_bboxes)
    return clusters


In [42]:
import os
import numpy as np
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import squareform

# Image size (needed to denormalize YOLO boxes)
IMG_WIDTH = 640
IMG_HEIGHT = 640

def read_yolo_labels(file_path):
    """
    Reads a YOLO label file and returns list of (class_id, [x1, y1, x2, y2]) in absolute pixels
    """
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:5])
            # Convert from YOLO format to [x1, y1, x2, y2]
            x1 = (x_center - w / 2) * IMG_WIDTH
            y1 = (y_center - h / 2) * IMG_HEIGHT
            x2 = (x_center + w / 2) * IMG_WIDTH
            y2 = (y_center + h / 2) * IMG_HEIGHT
            boxes.append((class_id, [x1, y1, x2, y2]))
    return boxes

def get_centroid(box):
    x1, y1, x2, y2 = box
    return [(x1 + x2) / 2, (y1 + y2) / 2]

def get_enclosing_box(cluster_boxes):
    x1s, y1s, x2s, y2s = zip(*cluster_boxes)
    return [min(x1s), min(y1s), max(x2s), max(y2s)]

def compute_custom_dist_matrix(boxes):
    n = len(boxes)
    dist_matrix = np.zeros((n, n))

    for i in range(n):
        x1_c, y1_c = get_centroid(boxes[i])
        w1 = boxes[i][2] - boxes[i][0]
        h1 = boxes[i][3] - boxes[i][1]
        for j in range(i + 1, n):
            x2_c, y2_c = get_centroid(boxes[j])
            w2 = boxes[j][2] - boxes[j][0]
            h2 = boxes[j][3] - boxes[j][1]

            dx = abs(x1_c - x2_c) - (w1 + w2) / 2
            dy = abs(y1_c - y2_c) - (h1 + h2) / 2
            distance = max(0, dx, dy)

            dist_matrix[i, j] = dist_matrix[j, i] = distance
    return dist_matrix

def cluster_boxes_per_image(label_file_path, eps=40, min_samples=1):
    class_to_boxes = {}

    labels = read_yolo_labels(label_file_path)
    for class_id, box in labels:
        class_to_boxes.setdefault(class_id, []).append(box)

    clusters_per_class = {}

    for class_id, boxes in class_to_boxes.items():
        if len(boxes) == 0:
            continue

        dist_matrix = compute_custom_dist_matrix(boxes)
        db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
        labels = db.fit_predict(dist_matrix)

        clusters = {}
        for lbl, box in zip(labels, boxes):
            if lbl == -1:
                continue  # noise
            clusters.setdefault(lbl, []).append(box)

        clusters_per_class[class_id] = [get_enclosing_box(c) for c in clusters.values()]

    return clusters_per_class

# Apply to all predicted labels
results_clusters = {}
for fname in sorted(os.listdir(results_labels_dir)):
    if not fname.endswith('.txt'):
        continue
    path = os.path.join(results_labels_dir, fname)
    clusters = cluster_boxes_per_image(path, eps=40)  # adjust `eps` based on spacing
    results_clusters[fname] = clusters

dataset_clusters = {}
for fname in sorted(os.listdir(data_labels_dir)):
    if not fname.endswith('.txt'):
        continue
    path = os.path.join(data_labels_dir, fname)
    clusters = cluster_boxes_per_image(path, eps=40)  # adjust `eps` based on spacing
    dataset_clusters[fname] = clusters



# Print results
# for fname, clusters in results_clusters.items():
#     print(f"File: {fname}")
#     for class_id, boxes in clusters.items():
#         print(f"  Class ID: {class_id}, Enclosing Boxes: {boxes}")
#     print()

save_dir = "/home/francescobarcherinii/Scrivania/FrancescoUni/Esami/unipi/1year2sem/IS/project/AI-waste-detection/datasets/extracted/RoboFlow-2_relabel/test/labels_clusters"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save results with format of labels tolo files
for fname, clusters in dataset_clusters.items():
    output_path = os.path.join(save_dir, fname)
    with open(output_path, 'w') as f:
        # if there is more than one class, write fname
        if len(clusters) > 1:
            print(f"Multiple classes found in {fname}, writing to file.")
            for class_id, boxes in clusters.items():
                print(f"  Class ID: {class_id}, Enclosing Boxes: {len(boxes)}")
                    
        for class_id, boxes in clusters.items():
            for box in boxes:
                x1, y1, x2, y2 = box
                x_center = (x1 + x2) / 2 / IMG_WIDTH
                y_center = (y1 + y2) / 2 / IMG_HEIGHT
                width = (x2 - x1) / IMG_WIDTH
                height = (y2 - y1) / IMG_HEIGHT
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


Multiple classes found in 0289980c-R_3443_jpg.rf.53ee0057dc1293e26a34c33fbe2a65a1.txt, writing to file.
  Class ID: 5, Enclosing Boxes: 1
  Class ID: 3, Enclosing Boxes: 1
Multiple classes found in 0d6885d5-R_621_jpg.rf.6e8c0572cac13b00e6357b595a2b486e.txt, writing to file.
  Class ID: 3, Enclosing Boxes: 1
  Class ID: 2, Enclosing Boxes: 1
Multiple classes found in 33a621f7-R_1264_jpg.rf.1128cd1e9a0d5d5c2dc3bd88c408e9d4.txt, writing to file.
  Class ID: 5, Enclosing Boxes: 1
  Class ID: 3, Enclosing Boxes: 1
Multiple classes found in 5af38683-R_1015_jpg.rf.5285b4a43213e5e9e2724bd92cb79711.txt, writing to file.
  Class ID: 1, Enclosing Boxes: 2
  Class ID: 5, Enclosing Boxes: 2
Multiple classes found in 95a1342f-R_795_jpg.rf.946f3c65d1bc9ddabfe1df95bbe758f2.txt, writing to file.
  Class ID: 3, Enclosing Boxes: 1
  Class ID: 5, Enclosing Boxes: 2
Multiple classes found in d67d51aa-R_830_jpg.rf.50aa8fc0a55a8c24137c4ec36a300cf8.txt, writing to file.
  Class ID: 5, Enclosing Boxes: 3
  Cla

In [56]:
# compute the IoU for two bounding boxes
def compute_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    if inter_x_min >= inter_x_max or inter_y_min >= inter_y_max:
        return 0.0

    intersection_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou

def compute_map50(dataset_clusters, predicted_clusters, iou_threshold=0.5):
    total_iou = 0
    total_count = 0

    for fname in dataset_clusters:
        if fname not in predicted_clusters:
            continue

        gt_per_class = dataset_clusters[fname]
        pred_per_class = predicted_clusters[fname]

        for class_id in gt_per_class:
            gt_boxes = gt_per_class[class_id]
            pred_boxes = pred_per_class.get(class_id, [])

            for gt_box in gt_boxes:
                best_iou = 0.0
                for pred_box in pred_boxes:
                    iou = compute_iou(gt_box, pred_box)
                    if iou >= iou_threshold:
                        best_iou = max(best_iou, iou)
                print(best_iou)
                if best_iou > 0:
                    total_iou += best_iou
                total_count += 1

    if total_count == 0:
        return 0.0
    return total_iou / total_count

print(compute_map50(dataset_clusters, results_clusters, iou_threshold=0.5))

0.8663080786538345
0.9049461100275739
0.94281473896677
0.0
0.0
0.918553048152468
0.9026827420664248
0.7871386694379878
0.838635126980605
0.5164331007213562
0.8299116067037677
0.0
0.9313811537888161
0.9203568033697959
0.9308249369192599
0.8602062961852176
0.7754648403129618
0.8802875015807228
0.750213268613537
0.7581247872814944
0.9216490357104766
0.8113469617243985
0.8097818959041749
0.0
0.0
0.9567702119908879
0.9463591803312081
0.8027750018916372
0.8394715073961615
0.9097398723111709
0.837102319558761
0.8978031050192058
0.0
0.8343985836588385
0.9241450581294731
0.8616276928498683
0.8967977633920461
0.6733457721621763
0.8943234666303674
0.8878052621689673
0.8687399123315048
0.9732335587745101
0.7962061706509461
0.6813592134285128
0.8771600898763262
0.7540230414977468
0.8259053384351533
0.9876614692243948
0.7816907481679826
0.9023053241588637
0.7859164310380397
0.0
0.9067455822517777
0.0
0.0
0.9388349193389801
0.8951500624031202
0.9832548237584101
0.7943977626337203
0.6035573610491752
0